# Semi-supervised Self-learning Classification with mnist Dataset

In [1]:
import shutil
import numpy as np
import pandas as pd
import tensorflow as tf
print(tf.__version__)

1.12.0


## Get data

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [3]:
print("x_train.shape = {}".format(x_train.shape))
print("y_train.shape = {}".format(y_train.shape))
print("x_test.shape = {}".format(x_test.shape))
print("y_test.shape = {}".format(y_test.shape))

x_train.shape = (60000, 28, 28)
y_train.shape = (60000,)
x_test.shape = (10000, 28, 28)
y_test.shape = (10000,)


In [4]:
HEIGHT = 28
WIDTH = 28
NCLASSES = 10

In [5]:
y_train = np.eye(N = NCLASSES)[y_train]
y_test = np.eye(N = NCLASSES)[y_test]

In [6]:
print("x_train.shape = {}".format(x_train.shape))
print("y_train.shape = {}".format(y_train.shape))
print("x_test.shape = {}".format(x_test.shape))
print("y_test.shape = {}".format(y_test.shape))

x_train.shape = (60000, 28, 28)
y_train.shape = (60000, 10)
x_test.shape = (10000, 28, 28)
y_test.shape = (10000, 10)


## Create fully supervised model for comparison

In [7]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x = {"image": x_train},
    y = y_train,
    batch_size = 100,
    num_epochs = None,
    shuffle = True,
    queue_capacity = 5000)

eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x = {"image": x_test},
    y = y_test,
    batch_size = 100,
    num_epochs = 1,
    shuffle = False,
    queue_capacity = 5000)

In [8]:
def linear_model(img, mode, hparams):
    X = tf.reshape(tensor = img, shape = [-1,HEIGHT * WIDTH]) # flatten
    ylogits = tf.layers.dense(inputs = X, units = NCLASSES, activation = None)
    return ylogits, NCLASSES

def dnn_model(img, mode, hparams):
    X = tf.reshape(tensor = img, shape = [-1, HEIGHT * WIDTH]) # flatten
    h1 = tf.layers.dense(inputs = X, units = 300, activation = tf.nn.relu)
    h2 = tf.layers.dense(inputs = h1, units = 100, activation = tf.nn.relu)
    h3 = tf.layers.dense(inputs = h2, units = 30, activation = tf.nn.relu)
    ylogits = tf.layers.dense(inputs = h3, units = NCLASSES, activation = None)
    return ylogits, NCLASSES

def dnn_dropout_model(img, mode, hparams):
    dprob = hparams.get("dprob", 0.1)

    X = tf.reshape(tensor = img, shape = [-1, HEIGHT * WIDTH]) #flatten
    h1 = tf.layers.dense(inputs = X, units = 300, activation = tf.nn.relu)
    h2 = tf.layers.dense(inputs = h1, units = 100, activation = tf.nn.relu)
    h3 = tf.layers.dense(inputs = h2, units = 30, activation = tf.nn.relu)
    h3d = tf.layers.dropout(inputs = h3, rate = dprob, training = (mode == tf.estimator.ModeKeys.TRAIN)) # only dropout when training
    ylogits = tf.layers.dense(inputs = h3d, units = NCLASSES, activation = None)
    return ylogits, NCLASSES

def cnn_model(img, mode, hparams):
    ksize1 = hparams.get("ksize1", 5)
    ksize2 = hparams.get("ksize2", 5)
    nfil1 = hparams.get("nfil1", 10)
    nfil2 = hparams.get("nfil2", 20)
    dprob = hparams.get("dprob", 0.25)

    c1 = tf.layers.conv2d(inputs = img, filters = nfil1,
                          kernel_size = ksize1, strides = 1, # ?x28x28x10
                          padding = "same", activation = tf.nn.relu)
    p1 = tf.layers.max_pooling2d(inputs = c1, pool_size = 2, strides = 2) # ?x14x14x10
    c2 = tf.layers.conv2d(inputs = p1, filters = nfil2,
                          kernel_size = ksize2, strides = 1, 
                          padding = "same", activation = tf.nn.relu)
    p2 = tf.layers.max_pooling2d(inputs = c2, pool_size = 2, strides = 2) # ?x7x7x20
    
    outlen = p2.shape[1] * p2.shape[2] * p2.shape[3] #980
    p2flat = tf.reshape(tensor = p2, shape = [-1, outlen]) # flattened

    # Apply batch normalization
    if hparams["batch_norm"]:
        h3 = tf.layers.dense(inputs = p2flat, units = 300, activation = None)
        h3 = tf.layers.batch_normalization(x = h3, training = (mode == tf.estimator.ModeKeys.TRAIN)) #only batchnorm when training
        h3 = tf.nn.relu(x = h3)
    else:    
        h3 = tf.layers.dense(inputs = p2flat, units = 300, activation = tf.nn.relu)
    
    # Apply dropout
    h3d = tf.layers.dropout(inputs = h3, rate = dprob, training = (mode == tf.estimator.ModeKeys.TRAIN))

    ylogits = tf.layers.dense(inputs = h3d, units = NCLASSES, activation = None)
        
    # Apply batch normalization once more
    if hparams["batch_norm"]:
         ylogits = tf.layers.batch_normalization(x = ylogits, training = (mode == tf.estimator.ModeKeys.TRAIN))

    return ylogits, NCLASSES

In [9]:
def image_classifier(features, labels, mode, params):
    print("\nfeatures = \n{}".format(features))
    print("labels = \n{}".format(labels))
    print("mode = \n{}".format(mode))
    print("params = \n{}".format(params))
    
    model_functions = {
        "linear":linear_model,
        "dnn":dnn_model,
        "dnn_dropout":dnn_dropout_model,
        "cnn":cnn_model}
    
    model_function = model_functions[params["model"]]    
    
    ylogits, nclasses = model_function(features["image"], mode, params)
    print("ylogits = \n{}".format(ylogits))
    probabilities = tf.nn.softmax(logits = ylogits) # shape = (current_batch_size, NCLASSES)
    print("probabilities = \n{}".format(probabilities))
    class_ids = tf.cast(x = tf.argmax(input = probabilities, axis = 1), dtype = tf.uint8) # shape = (current_batch_size,)
    print("class_ids = \n{}".format(class_ids))
    
    if mode == tf.estimator.ModeKeys.TRAIN or mode == tf.estimator.ModeKeys.EVAL:
        loss = tf.reduce_mean(input_tensor = tf.nn.softmax_cross_entropy_with_logits_v2(logits = ylogits, labels = labels))
        eval_metric_ops = {"accuracy": tf.metrics.accuracy(labels = tf.argmax(input = labels, axis = 1), predictions = class_ids)}
        if mode == tf.estimator.ModeKeys.TRAIN:
            # This is needed for batch normalization, but has no effect otherwise
            update_ops = tf.get_collection(key = tf.GraphKeys.UPDATE_OPS)
            with tf.control_dependencies(update_ops):
                train_op = tf.contrib.layers.optimize_loss(
                    loss = loss, 
                    global_step = tf.train.get_global_step(),
                    learning_rate = params["learning_rate"], 
                    optimizer = "Adam")
        else:
            train_op = None
    else:
        loss = None
        train_op = None
        eval_metric_ops = None
 
    return tf.estimator.EstimatorSpec(
                mode = mode,
                predictions = {"probabilities": probabilities, "class_ids": class_ids},
                loss = loss,
                train_op = train_op,
                eval_metric_ops = eval_metric_ops,
                export_outputs = {"classes": tf.estimator.export.PredictOutput({"probabilities": probabilities, "class_ids": class_ids})})

In [10]:
def serving_input_fn():
    # Input will be rank 3
    feature_placeholders = {"image": tf.placeholder(dtype = tf.float64, shape = [None, HEIGHT, WIDTH])}
    # But model function requires rank 4
    features = {"image": tf.expand_dims(input = feature_placeholders["image"], axis = -1)} 
    return tf.estimator.export.ServingInputReceiver(features = features, receiver_tensors = feature_placeholders)

In [11]:
def train_and_evaluate(output_dir, hparams):
    tf.summary.FileWriterCache.clear() # ensure filewriter cache is clear for TensorBoard events file
    EVAL_INTERVAL = 60

    supervised_estimator = tf.estimator.Estimator(model_fn = image_classifier,
                                                  params = hparams,
                                                  config = tf.estimator.RunConfig(save_checkpoints_secs = EVAL_INTERVAL),
                                                  model_dir = output_dir)
    
    train_spec = tf.estimator.TrainSpec(input_fn = train_input_fn,
                                        max_steps = hparams["train_steps"])
    
    exporter = tf.estimator.LatestExporter("exporter", serving_input_fn)
    
    eval_spec = tf.estimator.EvalSpec(input_fn = eval_input_fn,
                                      steps = None,
                                      exporters = exporter,
                                      throttle_secs = EVAL_INTERVAL)
    
    tf.estimator.train_and_evaluate(supervised_estimator, train_spec, eval_spec)
    
    return supervised_estimator

In [12]:
hparams = {}
hparams["train_batch_size"] = 100
hparams["learning_rate"] = 0.01
hparams["train_steps"] = 1000
hparams["ksize1"] = 5
hparams["ksize2"] = 5
hparams["nfil1"] = 10
hparams["nfil2"] = 20
hparams["dprob"] = 0.1
hparams["batch_norm"] = False
hparams["model"] = "linear"

In [13]:
SUPERVISED_MODEL_DIR = "supervised_trained"
shutil.rmtree(path = SUPERVISED_MODEL_DIR, ignore_errors = True) # start fresh each time
supervised_estimator = train_and_evaluate(SUPERVISED_MODEL_DIR, hparams)

INFO:tensorflow:Using config: {'_model_dir': 'supervised_trained', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 60, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0xb3ccb7fd0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evalu

In [14]:
eval_metrics = supervised_estimator.evaluate(input_fn = eval_input_fn, steps = None)

INFO:tensorflow:Calling model_fn.

features = 
{'image': <tf.Tensor 'fifo_queue_DequeueUpTo:1' shape=(?, 28, 28) dtype=float64>}
labels = 
Tensor("fifo_queue_DequeueUpTo:2", shape=(?, 10), dtype=float64, device=/device:CPU:0)
mode = 
eval
params = 
{'train_batch_size': 100, 'learning_rate': 0.01, 'train_steps': 1000, 'ksize1': 5, 'ksize2': 5, 'nfil1': 10, 'nfil2': 20, 'dprob': 0.1, 'batch_norm': False, 'model': 'linear'}
ylogits = 
Tensor("dense/BiasAdd:0", shape=(?, 10), dtype=float64)
probabilities = 
Tensor("Softmax:0", shape=(?, 10), dtype=float64)
class_ids = 
Tensor("Cast:0", shape=(?,), dtype=uint8)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-03-15-20:11:24
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from supervised_trained/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-03-15-20:11:25
INFO:tensorflow:Saving dict for 

## Now create semi-supervised model

In [15]:
number_of_train_examples = x_train.shape[0]
print("number_of_train_examples = {}".format(number_of_train_examples))

number_of_train_examples = 60000


In [16]:
number_of_labeled_train_examples = int(number_of_train_examples * 0.05)
number_of_unlabeled_train_examples = number_of_train_examples - number_of_labeled_train_examples
print("number_of_labeled_train_examples = {} & number_of_unlabeled_train_examples = {}".format(number_of_labeled_train_examples, number_of_unlabeled_train_examples))

number_of_labeled_train_examples = 3000 & number_of_unlabeled_train_examples = 57000


In [17]:
semi_supervised_labeled_x_train_original_arr = x_train[0:number_of_labeled_train_examples]
semi_supervised_labeled_y_train_original_arr = y_train[0:number_of_labeled_train_examples]
semi_supervised_unlabeled_x_train_original_arr = x_train[number_of_labeled_train_examples:]

In [18]:
print("semi_supervised_labeled_x_train_original_arr.shape = {}".format(semi_supervised_labeled_x_train_original_arr.shape))
print("semi_supervised_labeled_y_train_original_arr.shape = {}".format(semi_supervised_labeled_y_train_original_arr.shape))
print("semi_supervised_unlabeled_x_train_original_arr.shape = {}".format(semi_supervised_unlabeled_x_train_original_arr.shape))

semi_supervised_labeled_x_train_original_arr.shape = (3000, 28, 28)
semi_supervised_labeled_y_train_original_arr.shape = (3000, 10)
semi_supervised_unlabeled_x_train_original_arr.shape = (57000, 28, 28)


## Create semi-supervised model using sparse labels

In [19]:
SEMI_SUPERVISED_MODEL_DIR = "semi_supervised_trained"

In [20]:
EVAL_INTERVAL = 30
semi_supervised_estimator = tf.estimator.Estimator(model_fn = image_classifier,
                                                   params = hparams,
                                                   config = tf.estimator.RunConfig(save_checkpoints_secs = EVAL_INTERVAL),
                                                   model_dir = SEMI_SUPERVISED_MODEL_DIR)

INFO:tensorflow:Using config: {'_model_dir': 'semi_supervised_trained', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 30, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1c6aa4e7b8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [21]:
confidence_threshold = 0.95

In [22]:
shutil.rmtree(path = SEMI_SUPERVISED_MODEL_DIR, ignore_errors = True) # start fresh each time

semi_supervised_labeled_x_train_arr = semi_supervised_labeled_x_train_original_arr
semi_supervised_labeled_y_train_arr = semi_supervised_labeled_y_train_original_arr
semi_supervised_unlabeled_x_train_arr = semi_supervised_unlabeled_x_train_original_arr

new_labeled_x_train_arr = np.zeros([1])

accuracy = 0.000001
old_accuracy = 0.0

loop_counter = 0
print("\nloop_counter = {}, number_of_labeled_examples = {}, number_of_unlabeled_examples = {}\n".format(loop_counter, semi_supervised_labeled_x_train_arr.shape[0], semi_supervised_unlabeled_x_train_arr.shape[0]))
# Train on currently labeled data
train_input_fn = tf.estimator.inputs.numpy_input_fn(x = {"image": semi_supervised_labeled_x_train_arr}, 
                                                    y = semi_supervised_labeled_y_train_arr, 
                                                    batch_size = 32, 
                                                    num_epochs = None, 
                                                    shuffle = True)

semi_supervised_estimator.train(input_fn = train_input_fn, 
                                steps = 2000)


# Check evaluation metrics on held out evaluation set now that training is over
eval_metrics = semi_supervised_estimator.evaluate(input_fn = eval_input_fn, 
                                                  steps = None)

old_accuracy = accuracy
accuracy = eval_metrics["accuracy"]

# # Now predict from the unlabeled set
# predict_input_fn = tf.estimator.inputs.numpy_input_fn(x = {"image": semi_supervised_unlabeled_x_train_arr}, 
#                                                       y = None, 
#                                                       batch_size = 512, 
#                                                       num_epochs = 1, 
#                                                       shuffle = False)

# predictions = [prediction for prediction in semi_supervised_estimator.predict(input_fn = predict_input_fn)]

# # Get the probabilities from the prediction list generated from the estimator
# probabilities = np.array(object = [prediction["probabilities"] for prediction in predictions]) # shape = (semi_supervised_unlabeled_x_train_arr.shape[0], NUM_CLASSES)

# # Check if our predictions are above the confidence threshold
# confidence_condition = np.amax(a = probabilities, axis = 1) > confidence_threshold # shape = (semi_supervised_unlabeled_x_train_arr.shape[0],)

# # Create array of the confidently prediction examples combining their features with the predicted probabilities
# new_labeled_x_train_arr = semi_supervised_unlabeled_x_train_arr[confidence_condition]
# new_labeled_y_train_arr = probabilities[confidence_condition]

# semi_supervised_labeled_x_train_arr = np.concatenate(seq = [semi_supervised_labeled_x_train_arr, new_labeled_x_train_arr], axis = 0)
# semi_supervised_labeled_y_train_arr = np.concatenate(seq = [semi_supervised_labeled_y_train_arr, new_labeled_y_train_arr], axis = 0)

# # Remove the confident predictions leaving only the unconfident predictions to go another round through the loop
# semi_supervised_unlabeled_x_train_arr = semi_supervised_unlabeled_x_train_arr[~confidence_condition]


loop_counter = 0, number_of_labeled_examples = 3000, number_of_unlabeled_examples = 57000

INFO:tensorflow:Calling model_fn.

features = 
{'image': <tf.Tensor 'random_shuffle_queue_DequeueMany:1' shape=(32, 28, 28) dtype=float64>}
labels = 
Tensor("random_shuffle_queue_DequeueMany:2", shape=(32, 10), dtype=float64, device=/device:CPU:0)
mode = 
train
params = 
{'train_batch_size': 100, 'learning_rate': 0.01, 'train_steps': 1000, 'ksize1': 5, 'ksize2': 5, 'nfil1': 10, 'nfil2': 20, 'dprob': 0.1, 'batch_norm': False, 'model': 'linear'}
ylogits = 
Tensor("dense/BiasAdd:0", shape=(32, 10), dtype=float64)
probabilities = 
Tensor("Softmax:0", shape=(32, 10), dtype=float64)
class_ids = 
Tensor("Cast:0", shape=(32,), dtype=uint8)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into semi_supervised_tra

In [23]:
predict_input_fn = tf.estimator.inputs.numpy_input_fn(x = {"image": semi_supervised_unlabeled_x_train_arr}, 
                                                      y = None, 
                                                      batch_size = 512, 
                                                      num_epochs = 1, 
                                                      shuffle = False)

In [24]:
gen = semi_supervised_estimator.predict(input_fn = predict_input_fn)

In [25]:
predictions = [prediction for prediction in semi_supervised_estimator.predict(input_fn = predict_input_fn)]

INFO:tensorflow:Calling model_fn.

features = 
{'image': <tf.Tensor 'fifo_queue_DequeueUpTo:1' shape=(?, 28, 28) dtype=float64>}
labels = 
None
mode = 
infer
params = 
{'train_batch_size': 100, 'learning_rate': 0.01, 'train_steps': 1000, 'ksize1': 5, 'ksize2': 5, 'nfil1': 10, 'nfil2': 20, 'dprob': 0.1, 'batch_norm': False, 'model': 'linear'}
ylogits = 
Tensor("dense/BiasAdd:0", shape=(?, 10), dtype=float64)
probabilities = 
Tensor("Softmax:0", shape=(?, 10), dtype=float64)
class_ids = 
Tensor("Cast:0", shape=(?,), dtype=uint8)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from semi_supervised_trained/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [26]:
probabilities = np.array(object = [prediction["probabilities"] for prediction in predictions])

In [27]:
probabilities.shape

(57000, 10)

In [28]:
probabilities[0,:]

array([3.64081262e-03, 6.88067167e-10, 3.52739454e-03, 1.47519308e-08,
       4.10573518e-03, 1.65267933e-03, 5.16866599e-05, 2.24661840e-02,
       2.16985118e-04, 9.64338507e-01])

In [29]:
confidence_condition = np.amax(a = probabilities, axis = 1) > confidence_threshold

In [30]:
confidence_condition[0]

True

In [31]:
new_labeled_x_train_arr = semi_supervised_unlabeled_x_train_arr[confidence_condition]
new_labeled_y_train_arr = probabilities[confidence_condition]

In [32]:
new_labeled_x_train_arr.shape

(43477, 28, 28)

In [33]:
new_labeled_y_train_arr.shape

(43477, 10)

In [34]:
semi_supervised_labeled_x_train_arr.shape

(3000, 28, 28)

In [35]:
semi_supervised_labeled_y_train_arr.shape

(3000, 10)

In [44]:
a = np.random.rand(50000, 10)
b = np.random.rand(1300, 10)

In [ ]:
c = np.concatenate(seq = [a, b], axis = 0)
c.shape

In [ ]:
# np.concatenate(seq = [semi_supervised_labeled_y_train_arr, new_labeled_y_train_arr], axis = 0)

In [ ]:
# semi_supervised_labeled_x_train_arr = np.concatenate(seq = [semi_supervised_labeled_x_train_arr, new_labeled_x_train_arr], axis = 0)
# semi_supervised_labeled_y_train_arr = np.concatenate(seq = [semi_supervised_labeled_y_train_arr, new_labeled_y_train_arr], axis = 0)

In [ ]:
# semi_supervised_labeled_x_train_arr = np.stack(arrays = [semi_supervised_labeled_x_train_arr, new_labeled_x_train_arr], axis = 0)
# semi_supervised_labeled_y_train_arr = np.stack(arrays = [semi_supervised_labeled_y_train_arr, new_labeled_y_train_arr], axis = 0)

In [ ]:
# semi_supervised_labeled_y_train_arr = np.concatenate([semi_supervised_labeled_y_train_arr, new_labeled_y_train_arr], axis = 0)

In [ ]:
# semi_supervised_labeled_y_train_arr = np.array([semi_supervised_labeled_y_train_arr, new_labeled_y_train_arr])

In [ ]:
# shutil.rmtree(path = SEMI_SUPERVISED_MODEL_DIR, ignore_errors = True) # start fresh each time

# semi_supervised_labeled_x_train_arr = semi_supervised_labeled_x_train_original_arr
# semi_supervised_labeled_y_train_arr = semi_supervised_labeled_y_train_original_arr
# semi_supervised_unlabeled_x_train_arr = semi_supervised_unlabeled_x_train_original_arr

# new_labeled_x_train_arr = np.zeros([1])

# accuracy = 0.000001
# old_accuracy = 0.0

# loop_counter = 0
# while semi_supervised_unlabeled_x_train_arr.shape[0] > 0 and new_labeled_x_train_arr.shape[0] > 0 and accuracy > old_accuracy:
#     print("\nloop_counter = {}, number_of_labeled_examples = {}, number_of_unlabeled_examples = {}\n".format(loop_counter, semi_supervised_labeled_x_train_arr.shape[0], semi_supervised_unlabeled_x_train_arr.shape[0]))
#     # Train on currently labeled data
#     train_input_fn = tf.estimator.inputs.numpy_input_fn(x = {"image": semi_supervised_labeled_x_train_arr}, 
#                                                         y = semi_supervised_labeled_y_train_arr, 
#                                                         batch_size = 32, 
#                                                         num_epochs = None, 
#                                                         shuffle = True)

#     semi_supervised_estimator.train(input_fn = train_input_fn, 
#                                     steps = 2000)


#     # Check evaluation metrics on held out evaluation set now that training is over
#     eval_metrics = semi_supervised_estimator.evaluate(input_fn = eval_input_fn, 
#                                                       steps = None)
    
#     old_accuracy = accuracy
#     accuracy = eval_metrics["accuracy"]

#     # Now predict from the unlabeled set
#     predict_input_fn = tf.estimator.inputs.numpy_input_fn(x = {"image": semi_supervised_unlabeled_x_train_arr}, 
#                                                           y = None, 
#                                                           batch_size = 512, 
#                                                           num_epochs = 1, 
#                                                           shuffle = False)

#     predictions = [prediction for prediction in semi_supervised_estimator.predict(input_fn = predict_input_fn)]

#     # Get the probabilities from the prediction list generated from the estimator
#     probabilities = np.array(object = [prediction["probabilities"] for prediction in predictions])

#     # Check if our predictions are above the confidence threshold
#     confidence_condition = np.amax(a = probabilities, axis = 1) > confidence_threshold

#     # Create array of the confidently prediction examples combining their features with the predicted probabilities
#     new_labeled_x_train_arr = semi_supervised_unlabeled_x_train_arr[confidence_condition]
#     new_labeled_y_train_arr = probabilities[confidence_condition]

#     semi_supervised_labeled_x_train_arr = np.concatenate(seq = [semi_supervised_labeled_x_train_arr, new_labeled_x_train_arr], axis = 0)
#     semi_supervised_labeled_y_train_arr = np.concatenate(seq = [semi_supervised_labeled_y_train_arr, new_labeled_y_train_arr], axis = 0)

#     # Remove the confident predictions leaving only the unconfident predictions to go another round through the loop
#     semi_supervised_unlabeled_x_train_arr = semi_supervised_unlabeled_x_train_arr[~confidence_condition]
    
#     loop_counter += 1